In [ ]:
# Prototype notebook to organize the images. This will be converted to a regular python script.

In [ ]:
import core as co

## Hide warnings and check TF version

In [ ]:
co.ignore_warnings()
print(co.versions())

# Find images in directory

In [ ]:
imgfiles = co.get_images_recursive("input/")
imgdata = co.get_images_metadata(imgfiles)
imgdata

## Load model with class names, and check architecture

In [ ]:
(model, class_names) = co.load_model('model01')
print(class_names)
model.summary()

## Predict on new data

In [ ]:
img_size = co.IMG_SIZE
imgdata['pred_class'] = None
imgdata['pred_confidence'] = None

predictions_df = co.predict_df(model, imgdata, image_size=img_size, class_names=class_names)
predictions_df

In [ ]:
predictions_df[predictions_df['pred_class']=='screenshot'].shape

In [ ]:
predictions_df[predictions_df['pred_class']=='photo'].shape

In [ ]:
import pathlib
import os
import shutil
import hashlib
from PIL import Image

output_dir = os.path.abspath("output")

copied = 0
not_copied = 0
print("Copying images...")
for index, row in predictions_df.iterrows():
    src_file = row['path']
    
    # move this to metadata df
    img = Image.open(src_file)
    md5hash = hashlib.md5(img.tobytes()).hexdigest()
    img.close()
    
    file_ext = pathlib.Path(src_file).suffix
    dest_path = os.path.join(output_dir, row['pred_class']) #, row['cyear'])
    dest_file = os.path.join(dest_path, row['cdate'] + '-' + md5hash[0:7] + file_ext)
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    
    if not os.path.exists(dest_file):
        shutil.copy2(src_file, dest_file) # copy2 = copy with metadata
        copied+=1
    else:
        not_copied+=1

print(f"Copied {copied}/{len(predictions_df)} images. Not copied: {not_copied}")